<a href="https://colab.research.google.com/github/c-c-c-c/dm_integration/blob/master/myMecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 吉村インテグレーションステップ 

目的は、ドラマの構成/脚本として、「どのタイミングで」「どんな
出来事」が起こると良いかという示唆を見出すこと。

★目的変数
　　ドラマの初回視聴率からの上下動(%で、閾値を良い、悪い、
　　普通になるように３パターン準備)

★特徴量
　　-ドラマを恋愛、刑事、ヒューマンなどのカテゴリーに分ける
　　-さらに、ドラマをステージ(序盤、中盤、終盤)に分ける
　　-ドラマ名-ステージを行にした、単語×ドラマステージ行列を作成

★モデル
　　- 目的変数に対しロッソ回帰を行う。
　　- もしくは、教師なしのグルーピングを行う。(kmeans, トピックモデル)

In [44]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
aptitude is already the newest version (0.8.10-6ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-1)
git is already installed at the requested version (1:2.17.1-1ubuntu0.5)
make is already installed at the requested version (4.1-9.1ubuntu1)
curl is already installed at the requested version (7.58.0-2ubuntu3.8)
xz-utils is already installed at the requested version (5.2.2-1.3)
file is already installed at the requested version (1:5.32-2ubuntu0.3)
mecab is already installed at the requested version (0.996-5)
libmecab-dev is already installed at the requested version (0.996-5)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+mai

In [46]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

fatal: destination path 'mecab-ipadic-neologd' already exists and is not an empty directory.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :     tar => ok
[install-mecab-ipadic-NEologd] :     unxz => ok
[install-mecab-ipadic-NEologd] :     xargs => ok
[install-mecab-ipadic-NEologd] :     grep => ok
[install-mecab-ipadic-NEo

In [0]:
import joblib
import MeCab
import numpy as np
import pandas as pd
import re
import json
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso


from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC

## データの読み込み

In [0]:
# EPGデータ(手作業での修正中))
df = pd.read_excel("./drive/My Drive/0_インテグ作業/data/EPG_checking0212.xlsx")

In [0]:
f= open("./drive/My Drive/0_インテグ作業/data/drama_category0214.json", 'r')

#ココ重要！！
drama_category_dic = json.load(f) #JSON形式で読み込む



In [0]:
f= open("./drive/My Drive/0_インテグ作業/data/drama_win_lose.json", 'r')

#ココ重要！！
drama_win_lose_dic = json.load(f) #JSON形式で読み込む



In [51]:
df.columns

Index(['Unnamed: 0', 'drama_key', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'drama_title', 'sharp_title', 'sharp_cnt', 'sharp_cnt_epg_regex',
       'start_time', 'sharp_epg', 'sharp_epg_hand_corrected', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [0]:
df["sharp_epg_tknz"] = np.nan

In [0]:
# ゴミ除去

def removeTrash (text):
    import re

    result_text = text
    result_text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", result_text)
    result_text = re.sub(r"番組詳細|制作・著作|制作著作", "", result_text)
    result_text = re.sub(r"[!\(\)=『』～/]", "", result_text)
    result_text = re.sub(r"フジテレビ|日本テレビ|TBS|テレビ朝日|TBS|関西テレビ", "", result_text)
    
    result_text = re.sub(r"【公式.*?】", "", result_text)
    result_text = re.sub(r"\u3000", "", result_text)

    return result_text


In [54]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df['sharp_epg_hand_corrected']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df["sharp_epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# 手作業での前処理が完了してないので、完了したものだけにする
bool_list = [] 

for i in range(len(df["sharp_epg_hand_corrected"])):
    bool_list.append( type( df["sharp_epg_tknz"].iloc[i] ) != float )

In [0]:
df_notnull =  df[bool_list]

In [57]:
df_notnull["phys_cnt"] = np.nan
df_notnull["category"] = np.nan
df_notnull =  df_notnull.rename(columns={ 'Unnamed: 0' : 'sort_id' } )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
drama_category_dic.keys()

dict_keys(['love', 'plice'])

In [59]:
# 物理カウントとカテゴリーを加える
love_cnt = 0
police_cnt = 0
for tmp_key  in df_notnull["drama_key"].unique():
    cnt = 0

    if tmp_key in drama_category_dic["love"]:
        print("love☆")
        love_cnt += 1

        qry_l = " drama_key == @tmp_key"
        target_idx = df_notnull.query(qry_l).index
        df_notnull[ "category" ].loc [target_idx] = "love"


    if tmp_key in drama_category_dic["plice"]:
        police_cnt += 1
        qry_p = " drama_key == @tmp_key"
        target_idx = df_notnull.query(qry_p).index
        df_notnull[ "category" ].loc [target_idx] = "police"
        # print(police_cnt)

    for sort_id in  df_notnull[df_notnull["drama_key"] == tmp_key]['sort_id'].values:
        cnt  += 1

        qry = " sort_id == @sort_id"
        target_idx = df_notnull.query(qry).index
        df_notnull["phys_cnt"].loc[target_idx] = cnt

    print(tmp_key)


# df = pd.DataFrame( columns=['A','B'] )
# 行を作る
# tmp_se = pd.Series( [ i, i*i ], index=list_df.columns )
# # dfに行をレコードとして追加
# df = df.append( tmp_se, ignore_index=True )


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1910_CX_月21
1910_CX_火22
love☆
1910_CX_木22
1907_CX_木22
1904_CX_月21
love☆
1904_CX_火21
1904_CX_木22
1901_CX_月21
1901_CX_火21
1901_CX_木22
1810_CX_月21
1810_CX_火21
1810_CX_木22
1807_CX_月21
1807_CX_火21
1807_CX_木22
1804_CX_月21
1804_CX_火21
1804_CX_木22
love☆
1801_CX_月21
1801_CX_木22
1710_CX_月21
1710_CX_火21
1710_CX_木22
1707_CX_火21
1707_CX_木22
1707_CX_日21
1704_CX_月21
1704_CX_火21
1704_CX_木22
1704_CX_日21
1701_CX_月21
1701_CX_火21
1701_CX_木22
1701_CX_日21
1610_CX_月21
1610_CX_火21
1610_CX_木22
1610_CX_日21
1607_CX_月21
1607_CX_火22
1607_CX_日21
1604_CX_月21
1604_CX_火22
1604_CX_木22
1604_CX_土23
love☆
1601_CX_月21
1601_CX_火22
1601_CX_水22
1601_CX_木22
love☆
1510_CX_月21
1510_CX_火22
1510_CX_水22
love☆
1510_CX_木22
love☆
1507_CX_月21
1507_CX_火22
1507_CX_水22
1507_CX_木22
1504_CX_月21
love☆
1504_CX_水22
1504_CX_木22
love☆
1501_CX_月21
1501_CX_火21
1501_CX_火22
1501_CX_水22
1501_CX_木22
1410_CX_月21
1410_CX_火21
1410_CX_火22
1410_CX_水22
1410_CX_木22
1407_CX_月21
1407_CX_火21
1407_CX_火22
1407_CX_水22
love☆
1407_CX_木22
1407_CX_土23
1404_CX_月21
1404

In [60]:
df_notnull[df_notnull["category"] == "love"].drama_key.unique()

array(['1910_CX_木22', '1904_CX_火21', '1801_CX_月21', '1601_CX_月21',
       '1510_CX_月21', '1510_CX_木22', '1507_CX_月21', '1504_CX_水22',
       '1501_CX_月21', '1407_CX_木22', '1404_CX_木22', '1401_CX_月21',
       '1210_CX_木22', '1110_CX_月21', '1110_CX_木22', '1107_CX_月21',
       '1104_CX_月21', '1101_CX_月21', '1010_CX_月21', '1007_CX_月21',
       '1004_CX_月21', '1004_CX_木22', '0907_CX_月21', '0907_CX_火22',
       '0804_CX_火21', '0804_CX_火22', '0801_CX_月21', '0707_CX_月21',
       '1910_EX_土23', '1904_EX_土23', '1901_EX_土23', '1810_EX_土23',
       '1804_EX_土23', '1501_EX_金23', '1004_EX_木21', '1907_NTV_水22',
       '1907_NTV_土22', '1810_NTV_水22', '1807_NTV_水22', '1807_NTV_土22',
       '1804_NTV_木23', '1801_NTV_日22', '1804_TBS_火22', '1801_TBS_火22'],
      dtype=object)

In [0]:
# tmp_key = "1807_TBS_日21"
# qry = "drama_key == @tmp_key & phys_cnt < 4 "
# str(df_notnull.query(qry)["sharp_epg_hand_corrected"].values)

In [0]:
# df = pd.DataFrame( columns=['A','B'] )
# 行を作る
# tmp_se = pd.Series( [ i, i*i ], index=list_df.columns )
# # dfに行をレコードとして追加
# df = df.append( tmp_se, ignore_index=True )


df_stage = pd.DataFrame( columns=['drama_key','drama_title','stages','epg_joined','drama_category','win_lose'] )

# df_notnull["drama_key"].

for tmp_key in df_notnull[df_notnull["category"] == "love"].drama_key.unique():

    win_or_lose = ""
    if tmp_key in drama_win_lose_dic["win"]:
        win_or_lose = "win"
    elif tmp_key in drama_win_lose_dic["draw"]:
        win_or_lose = "draw"
    elif tmp_key in drama_win_lose_dic["lose"]:
        win_or_lose = "lose"
    else:
        print("????")

        target_idx = df_stage.query(qry_wl).index
        df_notnull[ "category" ].loc [target_idx] = "police"
        # print(police_cnt)


    for tmp_stage in ["early","middle" ,"late"]:

        qry = ""
        if tmp_stage == "early":
            qry = "drama_key == @tmp_key & phys_cnt < 4 "

        elif tmp_stage == "middle":
            qry = "drama_key == @tmp_key & 4< phys_cnt < 7 "

        else:
            qry = "drama_key == @tmp_key & phys_cnt >= 7 "

        target_i =  df_notnull.query(qry)["sharp_epg_hand_corrected"]
        epg_joined = str(df_notnull.query(qry)["sharp_epg_hand_corrected"].values)

        tmp_se = pd.Series( [ tmp_key, "..title" ,tmp_stage, epg_joined  ,"love", win_or_lose ]  , index=df_stage.columns  )
        df_stage = df_stage.append( tmp_se, ignore_index=True )

# early, middle, lateで作る

# for i in range():
    




In [63]:
df_stage


,drama_key,drama_title,stages,epg_joined,drama_category,win_lose
0,1910_CX_木22,..title,early,['現在失業中の27歳独身・難波ユリカ(新木優子)は、見た目にも気を使い、コミュニケーション...,love,lose
1,1910_CX_木22,..title,middle,['マコチ(高良健吾)は、ユリカ(新木優子)のもとへ謝りに行ったものの、追い返されてしまう。...,love,lose
2,1910_CX_木22,..title,late,['ユリカ(新木優子)は、マコチ(高良健吾)から「好きだ」と告白され、もう一度彼と付き合うこ...,love,lose
3,1904_CX_火21,..title,early,['12年ぶりの再会は偶然か、それとも運命かーー。恋をあきらめた鮎川樹(松坂桃李)と、初恋相...,love,win
4,1904_CX_火21,..title,middle,['つぐみ(山本美月)が駅のホームから転落した。車いすから懸命に手を伸ばすが届かず、線路に落...,love,win
5,1904_CX_火21,..title,late,['松本に戻って3カ月、つぐみ(山本美月)はインテリアコーディネーターとして働きながら、実家...,love,win
6,1801_CX_月21,..title,early,"['クラゲを愛しすぎてしまった筋金入りの“クラゲオタク女子""が、とある兄弟と三角関係になり、...",love,lose
7,1801_CX_月21,..title,middle,['稲荷翔子(泉里香)が『天水館』の持ち主である千絵子(富山えり子)の母に直接、土地建物の売...,love,lose
8,1801_CX_月21,..title,late,"['修(工藤阿須加)から告白された月海(芳根京子)は大混乱。“尼～ず""に相談出来ない月海は、...",love,lose
9,1601_CX_月21,..title,early,['幼いころに女手ひとつで育ててくれた母を亡くした杉原音(有村架純)は、育ての親である林田雅...,love,draw


In [0]:
# 空の列を足す
df_stage["epg_tknz"] = np.nan

In [65]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df_stage['epg_joined']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df_stage["epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [66]:
df_stage["epg_tknz"]

0      [[', 現在, 失業, 中, 27, 歳, 独身, 難波, ユリカ, 見た目, 気, 使い...
1      [[', マコチ, ユリカ, もと, 謝り, 行っ, 追い返さ, れ, しまう, 失意, マ...
2      [[', ユリカ, マコチ, 好き, 告白, さ, れ, もう一度, 彼, 付き合う, こと...
3      [[', 12, 年, ぶり, 再会, 偶然, それとも, 運命, ーー, 恋, あきらめ,...
4      [[', つぐみ, 駅, ホーム, 転落, し, 車いす, 懸命, 手, 伸ばす, 届か, ...
5      [[', 松本, 戻っ, 3, カ月, つぐみ, インテリア, コーディネーター, 働き, ...
6      [[', クラゲ, 愛し, すぎ, しまっ, 筋金入り, クラゲ, オタク, 女子, ", ...
7      [[', 稲荷, 天水, 館, 持ち主, 母, 直接, 土地, 建物, 売買, 契約, 持ち...
8      [[', 工藤, 阿須, 加, 告白, さ, れ, 月, 海, 芳, 根, 大, 混乱, 尼...
9      [[', 幼い, ころ, 女手, ひとつ, 育て, くれ, 母, 亡くし, 音, 有村, 架...
10     [[', 音, 有村, 架, 純, 朝陽, 誘わ, れ, 屋上, 行く, 朝陽, 用意, し...
11     [[', 音, 有村, 架, 純, 朝陽, プロポーズ, さ, れる, しかし, 音, 朝陽...
12     [['\, u, 3000, 地元, 町内, 会, 長老, 葬儀, 出席, し, い, やが...
13     [['\, u, 3000, デート, し, 家族, 一, 橋, 寺, 住職, たち, 意見...
14     [['\, u, 3000, こと, 意識, する, よう, なる, それ, 恋, の, ど...
15     [[', 40, 歳, 独身, 恋愛, アプリ, 手がける, 会社, キャリアウーマン, デ...
16     [['「, 2018, FIFA, ワールドカップ, アジア, 2, 次, 予選, 日本, ...
17     [[', 心配, し, 温泉, 宿, 駆けつけ,

In [0]:
df_stage["win_lose_dummy"] = np.nan
for i in range(len( df_stage["win_lose"] )):

    if df_stage["win_lose"].iloc[i] == "win":
        df_stage["win_lose_dummy"].iloc[i] = 1

    elif df_stage["win_lose"].iloc[i] == "draw":
        df_stage["win_lose_dummy"].iloc[i] = 0

    elif df_stage["win_lose"].iloc[i] == "lose":
        df_stage["win_lose_dummy"].iloc[i] = -1
        
    else :
        print("???")
    # df_stage["win_lose_dummy"] = 

In [0]:
df_early = df_stage[df_stage["stages"] == "early"]

In [0]:
df_middle = df_stage[df_stage["stages"] == "middle"]

In [0]:
df_late = df_stage[df_stage["stages"] == "late"]

In [0]:
bow_vectorizer_early = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
Xe = bow_vectorizer_early.fit_transform(
    [str(i) for i in df_early["epg_tknz"].values]    
)

In [0]:
Ye = df_early["win_lose_dummy"]

In [107]:
pd.DataFrame(Xe.toarray(), columns=[ x[0] for x in sorted(bow_vectorizer_early.vocabulary_.items(), key=lambda x: x[1]) ])

,1,10,100,110,12,18,2,20,2003,2013,2018,27,29,3,30,3000,30001,30002,30002008,30002015,30006,32,33,35,4,40,44,5,50,6,60,6000,7,70,ak,ap,arata,asamoto,b,bl,c,ca,ceo,ecu,ela,f,jc,jmt,mkm,no,ok,pc,ring,smbc,sns,tap,u,you,あいさつ,あえて,あおり,あおる,あがめる,あきらめ,あきらめる,あきれ,あきれる,あきれ返る,あくる,あくる日,あぐね,あげ,あげよ,あしらう,あっ,あっさり,あてつけ,あなた,あの,あふれる,あまり,あまりに,あやしい,あやしむ,あらわ,あり,ある,あれ,あわせ,い,いい,いい加減,いう,いえ,いか,いかにも,いき,いきなり,いく,いくら,いけ,いたずら,いただき,いた事,いちゃつく,いっ,いっこうに,いったん,いっぱい,いつ,いつか,いつしか,いつも,いとこ,いま,いやらしい,いよいよ,いら立ち,いり,いる,いわゆる,う,うかがい,うそ,うた,うち,うっかり,うっとうしく,うまく,うれしい,うろうろ,うろつく,うんざり,え,えり,えれ,お,おい,おかしな,おく,おごり,おごる,おしゃべり,おじけづい,おせっかい,おっさん,おなか,おばさん,おびえる,おぼつかない,おり,おんぶ,お互い,お似合い,お供,お化け,お子さま,お母さん,お気にいり,お礼,お腹,お茶,お金,お陰,か,かえ,かかっ,かかる,かかわら,かけ,かける,かすか,かつて,かなえる,かばい,かばっ,からかう,からかわ,が,がい,がち,がむしゃら,き,きい,きちんと,きっかけ,きり,きれ,くぎ付け,くし,くす,くつろい,くる,くれ,くれる,けんか腰,げ,げき,こ,こう,こき使わ,こぎつけ,こぎつける,ここ,こじらせ,こだわら,こと,ことごとく,この,こぼし,こぼす,こらえよ,これ,これから,ころ,こんな,ご,ごめん,さ,さくら,させ,させよ,させる,さっそく,さびれ,さまざま,さみし,さやか,さよなら,さら,さらす,さらに,さわやか,ざわつく,し,しおり,しかし,しかも,したがっ,しっかり,しびれ,しまい,しまう,しまえ,しまお,しまっ,しゃがみ,しよ,しようが,しれ,...,道,道明寺,達,違い,違う,違和感,遠く,遣,適合,遭っ,遭遇,選ぶ,選ん,選手,遺産,避け,那奈,那覇,邪魔,邸,郎,部,部下,部分,部屋,部長,郷里,都,都合,都市,配っ,配属,配慮,配達,酒,酒造,酔っ,酔っ払っ,醜態,里,重い,重く,重ね,重ねる,重荷,重要,重視,野心,金,金券,金沢,金策,鉢合わせ,銭湯,鋼,鍵,鎌倉,鐘,長,長い,長倉,長椅子,長男,長老,長野,閉ざさ,閉じ込め,閉め,閉口,閉館,開い,開か,開き,開く,開け,開ける,開催,開幕,開店,開発,間,間男,間違え,関わり,関わる,関係,関口,防ぐ,阿須,降り,降りしきる,降ろし,限ら,院長,除け者,陥っ,陰,陵,陽,陽子,階,階下,際,障,隠し,隠し事,隠し子,隠し持つ,隠せ,隠れ,隣,隣人,雄,雄大,雅,集い,集う,集まっ,集中,雑木林,雑誌,離れ,離れ離れ,離婚,難波,難癖,難関,雨,雨宿り,雪が谷大塚,雫,雰囲気,電話,電車,青年,青木,静,非常勤,非難,面々,面し,面倒,面持ち,面接,面談,靴,韓国,音,音大,響,響く,頃,順番,順調,預かっ,預かる,頑張っ,頓挫,頭,頭文字,頭痛,頼っ,頼ま,頼み,頼む,頼ん,顔,顔なじみ,顔色,願う,願っ,顧問,顧客,風,風呂,風見,風邪,飛ば,飛ばさ,飛ばし,飛び出し,飛び出す,飛び込む,飛行機,食,食べ,食事,食品,食器,飲み,飲む,飲ん,飼っ,館,香炉,馬,馬場,馳天,馴れ初め,駅,駅前,駆けつける,駆け付ける,駆け出し,駆け寄る,駆け込み,駐車,騒ぎ,驚い,驚がく,驚き,驚く,驚愕,高く,高まる,高め,高山,高嶺,高揚,高校,高校生,高梨,高等,高級,高速,高飛車,高鳴る,鬼,魅力,魚,鮮明,鯉淵,鳴か,鳴っ,鳴り,鳴り響い,鳴る,鶴,麦,麻衣,黒,黙っ,鼻,０,１,２,３,４,５,７
0,2,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,2,0,0,5,1,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,10,0,1,0,0,0,0,0,0,0,0,0,3,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,0,1,0,0,0,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,0,0,17,0,0,0,0,0,0,1,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,8,1,1,0,0,0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,8,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0

In [0]:
# # idf値の確認

# idf = pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
#     .to_frame("idf") \

In [111]:

scaler = StandardScaler(with_mean=False)
clf_e = Lasso(alpha=0.1)

scaler.fit(Xe)
clf_e.fit(scaler.transform(Xe), Ye)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [115]:

result_df_early = pd.DataFrame(clf_e.coef_.T , index=bow_vectorizer_early.vocabulary_ )
result_df_early[ result_df_early[0] != 0].sort_values(0)

,0
事実,-0.135032
会計士,-0.124948
兄,-0.116330
名乗る,-0.077159
心から,-0.077143
好感,-0.072126
届い,-0.062754
桁違い,-0.047706
達,-0.019058
割り込ん,-0.016240


In [0]:
bow_vectorizer_middle = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
Xm = bow_vectorizer_middle.fit_transform(
    [str(i) for i in df_middle["epg_tknz"].values]    
)

In [0]:
Ym = df_middle["win_lose_dummy"]

In [118]:

scaler = StandardScaler(with_mean=False)
clf_m = Lasso(alpha=0.1)

scaler.fit(Xm)
clf_m.fit(scaler.transform(Xm), Ym)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [119]:

result_df_middle = pd.DataFrame(clf_m.coef_.T , index=bow_vectorizer_middle.vocabulary_ )
result_df_middle[ result_df_middle[0] != 0].sort_values(0)

,0
未来,-1.747915e-01
駆けつけ,-1.606980e-01
健康,-1.027281e-01
病巣,-8.128889e-02
連絡,-4.452648e-02
ビー,-4.048328e-02
旅行,-3.557297e-02
戻す,-2.725225e-02
胸,-1.052975e-02
では,-9.342622e-03


In [0]:
bow_vectorizer_late = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
Xl = bow_vectorizer_late.fit_transform(
    [str(i) for i in df_late["epg_tknz"].values]    
)

In [0]:
Yl = df_late["win_lose_dummy"]

In [123]:
scaler = StandardScaler(with_mean=False)
clf_l = Lasso(alpha=0.1)

scaler.fit(Xl)
clf_l.fit(scaler.transform(Xl), Yl)


Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [124]:

result_df_late = pd.DataFrame(clf_l.coef_.T , index=bow_vectorizer_late.vocabulary_ )
result_df_late[ result_df_late[0] != 0].sort_values(0)

,0
思い出し,-0.122085
悲鳴,-0.116473
閉め,-0.103213
弁当,-0.066601
肺,-0.057052
近,-0.034354
欠勤,-0.032113
かけ,-0.019675
すがり,-0.014400
尽くす,-0.013694


In [0]:
#####################
#
#以下、授業の残り
#
####################



## 形態素解析
---

MeCab を利用した形態素解析と、簡単な処理の例を示します。

### 簡単な例

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

CountVectorizer が生成する単語文書行列では、行 (横) 方向が1つの文書を表し、それぞれの列に単語の出現回数が格納されます (※単語文書行列と言った場合、列方向が文書を表す場合もあるので、他の文献を読む際は注意が必要です)。

どの列がどの単語と対応しているかを知るためには、vectorizer の vocabulary_ を参照します。

In [0]:
vectorizer.vocabulary_

新しい文書に対しては、transform で変換します。

なお、fit\_transform の際に出てこなかった単語 (= vocabulary_ にない単語) については、行列内に登場しません。

In [0]:
# transform も、リスト形式で与える必要があるので注意
X_new = vectorizer.transform([
    "Hello I like this pen"
])
X_new.toarray()

### 日本語データに適用
前処理済みの日本語データに適用してみます。DataFrame にスペース区切りのテキストを準備している場合、DataFrame の列を参照させたものをそのまま与えることができます。

In [0]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X

In [0]:
X[0].toarray()

↑この規模の単語文書行列になると、行列のサイズ (文書数x単語数) と比較し、値が入っている場所をほとんど無いことが分かります。

In [0]:
vectorizer.vocabulary_

## TF-IDF変換
---

TF-IDF 変換を行うと、単語の出現回数に、他の文書全体と比較した際の希少性 (レア度) で重みを付けることができます。

TF-IDF 変換済みの単語文書行列は、`TfidfVectorizer` で作成できます。

### 簡単な例

In [0]:
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:
df[""]

In [0]:
# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])
X.toarray()

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

他の文書でも出現している単語については、同じ出現回数でも重みが低くなっていることが分かります。各単語の重みは、`idf_` で見ることができます。確認してみましょう。

In [0]:
vectorizer.idf_

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(np.atleast_2d(vectorizer.idf_), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

デフォルトでは、各文書のノルムが1になるように正規化されています (norm="l2")。確認してみましょう。

In [0]:
np.linalg.norm(X.toarray(), axis=1)

### 日本語データに適用
CountVectorizer の場合と同様に、前処理済みの日本語データに適用してみます。

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X[0].toarray()

IDF 値の大きい単語・小さい単語を確認してみましょう。

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
    .to_frame("idf") \
    .sort_values("idf", ascending=False)

## テキスト分類
---
定量化したテキスト情報を使うことで、テキストをカテゴリー別に分類してみましょう

In [0]:
# テストデータ
df_test = pd.read_csv("dataset/wikipedia-test.txt", sep="\t")
df_test.head()

### 単語文書行列 (TF-IDF 適用済み) の作成
単語文章行列を作成する前に、まずは文章を形態素解析します。

In [0]:
mecab = MeCab.Tagger("-O wakati")

text_tokenized = []
for text in df_test['text']:
    text_tokenized.append(mecab.parse(text))
    
df_test['text_tokenized'] = text_tokenized

続いて、単語文章行列を学習データテストデータでそれぞれ生成します。<br>
この時、2つの単語文章行列は同じ次元数でなければならないので、`transform`を使用します

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X_test = vectorizer.transform(df_test["text_tokenized"])

### 分類器の適用・比較
単語文書行列ができたら、後はこれまでやってきた機械学習と同様です。

いくつかの分類器で、性能を比較してみましょう。

In [0]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### ロジスティック回帰

In [0]:
clf_lr = LogisticRegression(n_jobs=-1)
clf_lr.fit(X, df["category"])

推定結果は predict で得ることができます (他の分類器でも同様)。

In [0]:
clf_lr.predict(X_test)

In [0]:
y_test_pred = clf_lr.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

### SVM (線形カーネル)

In [0]:
clf_svc = LinearSVC()
clf_svc.fit(X, df["category"])

In [0]:
y_test_pred = clf_svc.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

### Random Forest

In [0]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(X, df["category"])

In [0]:
y_test_pred = clf_rf.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

## 【演習1】
上記いずれかのアルゴリズムのパラメータを変更して、分類精度を高めてみましょう。

In [0]:
# Enter your code here

### モデルの保存
`pickle` でも保存できますが、`joblib` を使い、ファイル名末尾に `.gz`, `.bz2` 等を指定すると、自動的に圧縮してくれます。読み込み時も同様、自動的に解凍して読み込んでくれます。

言語処理ではモデルが大容量になることが多いため、モデルの圧縮は重要です。

In [0]:
joblib.dump(clf_rf, "wikipedia_category_classifier.pkl.gz")

### モデル読み込みテスト

In [0]:
clf_rf_restored = joblib.load("wikipedia_category_classifier.pkl.gz")
clf_rf_restored

読み込んだモデルは、上記ですでに学習済みなので、`predict`で予測が行えます

In [0]:
clf_rf_restored.predict(X_test)